<h1 style='text-align:center; font-weight:600;'>Test Your Models with your Webcam</h1>

In [1]:
import tensorflow as tf
import cv2
import numpy as np
import mediapipe as mp
import os

In [2]:
model = tf.keras.models.load_model('Models/AbhayaV2.04_Corrected_Randomized.keras')

In [3]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [4]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z, res.visibility] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*4)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    
    return np.concatenate([pose, face, lh, rh])

In [5]:
# Converts video for feature recognition
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

In [6]:
def draw_styled_landmarks(image, result):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                              mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1),
                              )
    
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [8]:
actions =  np.array(['Are you Free Today', 'Can you repeat that please',
       'Congratulations', 'Help Me Please', 'how are you', 'I am fine',
       'I love you', 'no', 'Please come,Welcome', 'Talk slower please',
       'Thank You', 'What are you doing', 'What do you do',
       'What Happened','yes'])

In [11]:
sequence = []
sentence = []
threshold = 0.4


cap = cv2.VideoCapture(0)

try:
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, frame = cap.read()
            image, results = mediapipe_detection(frame, holistic)
            
            draw_styled_landmarks(image, results)
            
            keypoints = extract_keypoints(results)
            sequence.insert(0,keypoints)
            sequence = sequence[:30]
            
            if len(sequence) == 30:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                print(actions[np.argmax(res)])
             
            
            cv2.imshow('Webcam Feed', image)
            
            if cv2.waitKey(10) & 0xff == ord('q'):
                cap.release()
                cv2.destroyAllWindows()

except Exception as e:
    print(e)
    cap.release()
    cv2.destroyAllWindows()
            

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
What Happened
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
What Happened
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
What are you doing
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Please come,Welcome
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Please come,Welcome
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
What are you doing
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
What are you doing
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Please come,Welcome
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
What are you doing
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Please come,Welcome
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Please come,Welcome
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
What are you doing
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
What are you doing
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
What Happened
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Please come,Welcome
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Please come,Welcome
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Please come,Welcome
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


NameError: name 'q' is not defined